In [45]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import joblib
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [46]:
data_path = "./Data.csv"
df = pd.read_csv(data_path)

# df = df[['networkExp', 'bedwars_level', 'Experience', 'beds_broken_bedwars', 'beds_lost_bedwars', 'coins', 'deaths_bedwars', 'diamond_resources_collected_bedwars', 'emerald_resources_collected_bedwars', 'final_deaths_bedwars', 'final_kills_bedwars', 'games_played_bedwars', 'games_played_bedwars_1', 'kills_bedwars', 'losses_bedwars', 'void_final_deaths_bedwars',  'wins_bedwars','fkdr','wlr','bblr','fk_lev','bb_lev','kill_lev', "Cheat"]]
df = df[['karma', 
        'bedwars_loot_box',
        'all_timeBEDWARS__defensive', 'BEDWARS__offensive',
        'BEDWARS__support', 'Bedwars_openedChests',
        'beds_broken_bedwars', 'beds_lost_bedwars',
        'deaths_bedwars', 'diamond_resources_collected_bedwars',
        'emerald_resources_collected_bedwars', 'fall_deaths_bedwars',
        'final_deaths_bedwars', 'final_kills_bedwars',
        'games_played_bedwars', 'games_played_bedwars_1', 'kills_bedwars',
        'losses_bedwars', 'void_deaths_bedwars', 'void_final_deaths_bedwars',
        'void_final_kills_bedwars', 'void_kills_bedwars', 'wins_bedwars', "Cheat"
        ]]

In [47]:
# Cheatカラムを削除して、各レコードの合計を計算する
row_sums = df.drop('Cheat', axis=1).sum(axis=1)

# 分母が0になる可能性がある行のインデックスを取得
zero_division_indices = row_sums[row_sums == 0].index

# 10,000をその行の合計値で割った値を計算して保持する
divisors = pd.Series(0, index=df.index)  # 全ての行を0で初期化
divisors[zero_division_indices] = 0  # 分母が0になる行のみ0に設定
divisors[~divisors.index.isin(zero_division_indices)] = 10000 / row_sums[~row_sums.index.isin(zero_division_indices)]

# それぞれの行に対して計算した値を掛ける
df_scaled = df.drop('Cheat', axis=1).mul(divisors, axis=0)

# データフレームにCheatカラムを追加する
df_scaled['Cheat'] = df['Cheat']
print(df_scaled)

# 新しいデータベースとして保存する
# df_scaled.to_csv('ScaledData.csv', index=False)

            karma  bedwars_loot_box  all_timeBEDWARS__defensive  \
0     5991.990142          1.026905                         0.0   
1       75.949367         27.004219                         0.0   
2     9702.969049          0.630104                         0.0   
3     5876.654827          0.000000                         0.0   
4        0.000000          8.962868                         0.0   
...           ...               ...                         ...   
4673  2476.356758          0.000000                         0.0   
4674    30.491523          3.658983                         0.0   
4675   344.382264          2.869852                         0.0   
4676   117.435411          3.498076                         0.0   
4677   116.369279          0.000000                         0.0   

      BEDWARS__offensive  BEDWARS__support  Bedwars_openedChests  \
0              55.452865          2.053810              3.080715   
1              15.189873         25.316456             75.9

In [48]:
# df = pd.get_dummies(df)
# print("dummies")

X = df_scaled.drop('Cheat', axis=1)
y = df_scaled['Cheat']

# データの標準化
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# モデルの作成
model = RandomForestClassifier(n_estimators=2000, random_state=42, class_weight={0: 1, 1: 3})
model.fit(X_train, y_train)

# 訓練データの正解率
train_accuracy = model.score(X_train, y_train)
print(f'Training Accuracy: {train_accuracy}')

# テストデータの正解率
test_accuracy = model.score(X_test, y_test)
print(f'Testing Accuracy: {test_accuracy}')

# モデルと標準化器の保存
# joblib.dump(model, './models/Cheater.pkl')
# joblib.dump(scaler, './models/scaler.joblib')


Training Accuracy: 0.997327632282202
Testing Accuracy: 0.8942307692307693


In [49]:
joblib.dump(model, '../Cheater.pkl')
# joblib.dump(scaler, './models/scaler.joblib')

['../Cheater.pkl']

# snipe垢の条件

fkdr, wlr, bblrは低い

star n.levは30以下

sniper%が高い　（ターゲットが見つかるまで何回もｒｑするから）

fk/lev bb/lev kill/lev が高い　fk/levは7.5 bb/levは6.5 kill/levは20以上で高い

# cheatersデータ - snipe垢 = not sniper cheaters
snipe垢の条件外のcheaterを予測する。

# cubelifyのsniper%
cubelifyのsniper%を取得して、学習データとする